# Probando Librerías

In [1]:
import itertools
import unidecode

import numpy as np
import pandas as pd
import re

#NLTK
import nltk
from nltk.tokenize import RegexpTokenizer   
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.stem.porter import *
nltk.download('punkt')
nltk.download('stopwords')

#SPACY
import spacy
import es_core_news_sm

#STANZA
import stanza
stanza.download("es")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vanes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vanes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2021-07-30 13:27:30 INFO: Downloading default packages for language: es (Spanish)...
2021-07-30 13:27:31 INFO: File exists: C:\Users\vanes\stanza_resources\es\default.zip.
2021-07-30 13:27:36 INFO: Finished downloading models and saved to C:\Users\vanes\stanza_resources.


In [2]:
data = pd.read_json('dataset_es_dev.json', lines=True)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0417480,product_es_0873923,reviewer_es_0672978,1,"Malisimo, muy grande demasiado aparatoso y mal...",Mala compra,es,wireless
1,es_0180432,product_es_0713146,reviewer_es_0100858,1,No he recibido el pedido no la devolución,No lo he recibido,es,apparel
2,es_0144850,product_es_0356874,reviewer_es_0486447,1,"Tengo que buscar otro sistema, este no funcion...",Que no aprieta bien en el manillar,es,sports
3,es_0339629,product_es_0939832,reviewer_es_0894703,1,Utilicé las brocas de menor diámetro y se me d...,Brocas de mantequilla,es,home_improvement
4,es_0858362,product_es_0489066,reviewer_es_0887663,1,No me gusta su olor a viejo y aspecto malo,No me gusta,es,beauty


In [3]:
#Selecciono columnas de interés
data=data[['stars', 'review_title', 'review_body']]
#Concateno columnas
data['total_review']=data.review_body + ' ' + data.review_title
#Selecciono sólo 100 instancia para la prueba
data=data.iloc[:100]
print(data.shape)
data.head()

(100, 4)


,stars,review_title,review_body,total_review
0,1,Mala compra,"Malisimo, muy grande demasiado aparatoso y mal...","Malisimo, muy grande demasiado aparatoso y mal..."
1,1,No lo he recibido,No he recibido el pedido no la devolución,No he recibido el pedido no la devolución No l...
2,1,Que no aprieta bien en el manillar,"Tengo que buscar otro sistema, este no funcion...","Tengo que buscar otro sistema, este no funcion..."
3,1,Brocas de mantequilla,Utilicé las brocas de menor diámetro y se me d...,Utilicé las brocas de menor diámetro y se me d...
4,1,No me gusta,No me gusta su olor a viejo y aspecto malo,No me gusta su olor a viejo y aspecto malo No ...


In [4]:
#Defino stopwords con NLTK
stopwords = nltk.corpus.stopwords.words('spanish')
#Considermos incluir al listado de stopwords de nltk algunas palabras sin su acento.
stopwords_sin_acentos =  ['si', 'tambien', 'que', 'el', 'mio', 'mia', 'mios', 'mias', 'estas', 'esta', 'estamos', 'estais', 'estan', 'esta', 'estes', 
                         'esteis', 'esten', 'estare', 'estaras', 'estara', 'estareis', 'estaran', 'estaria', 'estarias', 'estariamos', 'estariais', 
                         'estarian', 'estuvieramos', 'estuviesemos', 'habeis',  'hayais','habre', 'habras', 'habra', 'habreis', 'habran',
                         'habria', 'habrias', 'habriamos', 'habriais', 'habrian', 'habia', 'habias', 'habiamos', 'habiais', 'habian', 'hubieramos', 
                         'hubiesemos', 'seais', 'sere', 'seras', 'sera', 'sereis', 'seran', 'seria', 'serias', 'seriamos', 'seriais', 'serian', 
                         'eramos', 'fueramos','fuesemos', 'teneis', 'tengais', 'tendre', 'tendras', 'tendra', 'tendreis', 'tendran', 'tendria', 'tendrias',
                         'tendriamos', 'tendriais', 'tendrian', 'tenia', 'tenias', 'teniamos', 'teniais', 'tenian', 'tuvieramos', 'tuviesemos']

#Consideremos incluir algunas palabras cuya función no aporta al modelo
stopwords_extra = ['vez', 'ademas','dia','dias','mes','meses','año','ano','lunes','martes','miercoles','jueves','viernes','sabado','domingo','ayer','hoy','mañana','manana',
                   'enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre','amazon', 'primero', 'segundo',
                   'tercero','cuarto','quinto', 'sexto','septimo','octavo','noveno','decimo','uno','dos','tres','cuatro','cinco','seis','siete','ocho','nueve','diez',
                   'cien','mil']
#Consideremos excluir de las stopwords algunos adverbios de cantidad /afirmación/negación
adv_cantidad = ['muy','mucho','muchos','mas','poco','nada','sin','tanto','ni','no','si']

#Definimos nuevas stopwords
stopwords = stopwords + stopwords_sin_acentos + stopwords_extra
stopwords = [word for word in stopwords if word not in adv_cantidad]
#stopwords

In [5]:
#Defino función para normalizar texto y generar tokens
def normalizar(reviews, stopwords, list_tokens):
    for i in range(len(reviews)):
        review = reviews[i] #selecciono el comentario
        #Elimino acentos
        review = unidecode.unidecode(str(review)) 
        #Elimino todo lo que no sea letras
        review = re.sub("[^a-zA-Z]"," ",review)
        #Elimino números
        review = re.sub("[0-9]"," ",review)   
        #Tokenizo
        review = nltk.word_tokenize(review)
        #Remuevo StopWords
        review = [palabra for palabra in review if  not palabra in stopwords] 
        #Elimino palabras con menos de 2 letras.
        review = [palabra for palabra in review if len(palabra)>2]
        # Agrego el resultado a la lista
        list_tokens.append(review) 

In [6]:
#Paso a minúsculas todas las palabras del corpus
data['total_review'] = data['total_review'].str.lower()

In [7]:
# Normalizo e incluyo al dataset una columna con los tokens generados
reviews=data.total_review.tolist()
reviews_tokens=[]
normalizar(reviews, stopwords, reviews_tokens)
data['review_tokens']=reviews_tokens
data.iloc[:10]

,stars,review_title,review_body,total_review,review_tokens
0,1,Mala compra,"Malisimo, muy grande demasiado aparatoso y mal...","malisimo, muy grande demasiado aparatoso y mal...","[malisimo, muy, grande, demasiado, aparatoso, ..."
1,1,No lo he recibido,No he recibido el pedido no la devolución,no he recibido el pedido no la devolución no l...,"[recibido, pedido, devolucion, recibido]"
2,1,Que no aprieta bien en el manillar,"Tengo que buscar otro sistema, este no funcion...","tengo que buscar otro sistema, este no funcion...","[buscar, sistema, funciona, muy, bien, abrazad..."
3,1,Brocas de mantequilla,Utilicé las brocas de menor diámetro y se me d...,utilicé las brocas de menor diámetro y se me d...,"[utilice, brocas, menor, diametro, doblaron, v..."
4,1,No me gusta,No me gusta su olor a viejo y aspecto malo,no me gusta su olor a viejo y aspecto malo no ...,"[gusta, olor, viejo, aspecto, malo, gusta]"
5,1,Bolas rotas,"Hola, después de recoger el paquete me ha lleg...","hola, después de recoger el paquete me ha lleg...","[hola, despues, recoger, paquete, llegado, bol..."
6,1,A parte de no poder elegir color...,"A parte de no poder elegir color, te envian el...","a parte de no poder elegir color, te envian el...","[parte, poder, elegir, color, envian, cesto, c..."
7,1,Tapa pantalla,Muy baja calidad de ahí si precio. Esta compra...,muy baja calidad de ahí si precio. esta compra...,"[muy, baja, calidad, ahi, precio, compra, supo..."
8,1,NO HE RECIBIDO EL PRODUCTO,El producto sale como entregado pero yo no he ...,el producto sale como entregado pero yo no he ...,"[producto, sale, entregado, recibido, nada, re..."
9,1,Una patata.,"No lo recomiendo, no se pega y acaba despegandose","no lo recomiendo, no se pega y acaba despegand...","[recomiendo, pega, acaba, despegandose, patata]"


In [8]:
data=data[['stars', 'total_review','review_tokens']]
data.head()

,stars,total_review,review_tokens
0,1,"malisimo, muy grande demasiado aparatoso y mal...","[malisimo, muy, grande, demasiado, aparatoso, ..."
1,1,no he recibido el pedido no la devolución no l...,"[recibido, pedido, devolucion, recibido]"
2,1,"tengo que buscar otro sistema, este no funcion...","[buscar, sistema, funciona, muy, bien, abrazad..."
3,1,utilicé las brocas de menor diámetro y se me d...,"[utilice, brocas, menor, diametro, doblaron, v..."
4,1,no me gusta su olor a viejo y aspecto malo no ...,"[gusta, olor, viejo, aspecto, malo, gusta]"


# Stemming NLTK

In [9]:
#Definimos función para encontrar el stem de cada palabra con el algoritmo snowball
def stemming_snowball(review_tokens, review_stemm):
    stemmer = SnowballStemmer('spanish')
    for rev in review_tokens:
        rev=[stemmer.stem(palabra) for palabra in rev]
        rev = " ".join(rev)
        review_stemm.append(rev)

In [10]:
#Definimos función para encontrar el stem de cada palabra con el algoritmo porter
def stemming_porter(review_tokens, review_stemm):
    stemmer = PorterStemmer()
    for rev in review_tokens:
        rev=[stemmer.stem(palabra) for palabra in rev]
        rev = " ".join(rev)
        review_stemm.append(rev)

In [11]:
review_stemm_snow=[]
stemming_snowball(data.review_tokens, review_stemm_snow)
data_stemm=data[['stars']]
data_stemm["rev_stem_snow"] = review_stemm_snow
data_stemm.head()

<ipython-input-11-5347a26ec078>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_stemm["rev_stem_snow"] = review_stemm_snow


,stars,rev_stem_snow
0,1,malisim muy grand demasi aparat mal protector ...
1,1,recib ped devolucion recib
2,1,busc sistem funcion muy bien abrazader llev ag...
3,1,utilic broc menor diametr dobl vari mantequill...
4,1,gust olor viej aspect mal gust


In [12]:
review_stemm_porter=[]
stemming_porter(data.review_tokens, review_stemm_porter)
data_stemm["rev_stem_porter"] = review_stemm_porter
data_stemm.head()

,stars,rev_stem_snow,rev_stem_porter
0,1,malisim muy grand demasi aparat mal protector ...,malisimo muy grand demasiado aparatoso mal pro...
1,1,recib ped devolucion recib,recibido pedido devolucion recibido
2,1,busc sistem funcion muy bien abrazader llev ag...,buscar sistema funciona muy bien abrazadera ll...
3,1,utilic broc menor diametr dobl vari mantequill...,utilic broca menor diametro doblaron varia man...
4,1,gust olor viej aspect mal gust,gusta olor viejo aspecto malo gusta


# Lemmatizar

In [13]:
data_lemma=data[['stars']]

## NLTK


In [14]:
def lemmatizar_nltk(rev_tokens, rev_lemma_nltk):
    wordnet_lemmatizer = WordNetLemmatizer()
    for rev in rev_tokens:
        rev=[wordnet_lemmatizer.lemmatize(palabra) for palabra in rev]
        rev = " ".join(rev)
        rev_lemma_nltk.append(rev)

In [15]:
rev_lemma_nltk=[]
lemmatizar_nltk(data.review_tokens, rev_lemma_nltk)
data_lemma["rev_lemma_nltk"] = rev_lemma_nltk
data_lemma.head()

<ipython-input-15-101dad4be4e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_lemma["rev_lemma_nltk"] = rev_lemma_nltk


,stars,rev_lemma_nltk
0,1,malisimo muy grande demasiado aparatoso mal pr...
1,1,recibido pedido devolucion recibido
2,1,buscar sistema funciona muy bien abrazadera ll...
3,1,utilice broca menor diametro doblaron varias m...
4,1,gusta olor viejo aspecto malo gusta


## Spacy

In [16]:
sp = spacy.load("es_core_news_sm")

In [17]:
def lemmatizar_spacy(review_tokens, rev_lemma_sp):
    sp = spacy.load("es_core_news_sm")
    for rev in range(len(review_tokens)):
        doc= sp(review_tokens[rev])
        rev_lemma=[palabra.lemma_ for palabra in doc]
        rev_lemma=" ".join(rev_lemma)
        rev_lemma_sp.append(rev_lemma)

In [18]:
def unir_palabras(fila):
    lista = fila['review_tokens']
    palabras_unidas = ( " ".join(lista))
    return palabras_unidas

In [19]:
data_lemma['review_tokens_unidos']= data.apply(unir_palabras, axis=1)
review_tokens_unidos=data_lemma.review_tokens_unidos.tolist()

In [20]:
rev_lemma_spacy=[]
lemmatizar_spacy(review_tokens_unidos, rev_lemma_spacy)
data_lemma["rev_lemma_spacy"] = rev_lemma_spacy
data_lemma.head()

,stars,rev_lemma_nltk,review_tokens_unidos,rev_lemma_spacy
0,1,malisimo muy grande demasiado aparatoso mal pr...,malisimo muy grande demasiado aparatoso mal pr...,malisimo mucho grande demasiado aparatoso mal ...
1,1,recibido pedido devolucion recibido,recibido pedido devolucion recibido,recibido pedido devolucion recibido
2,1,buscar sistema funciona muy bien abrazadera ll...,buscar sistema funciona muy bien abrazadera ll...,buscar sistema funcionar mucho bien abrazadera...
3,1,utilice broca menor diametro doblaron varias m...,utilice brocas menor diametro doblaron varias ...,utilizar broca menor diametro doblar varios ma...
4,1,gusta olor viejo aspecto malo gusta,gusta olor viejo aspecto malo gusta,gustar olor viejo aspecto malo gustar


## Stanza

In [21]:
def lemmatizar(rev_tokens, rev_lemma_st):
    nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')
    for rev in range(len(rev_tokens)):
        doc=nlp(rev_tokens[rev])
        rev_lemma=[palabra.lemma for oracion in doc.sentences for palabra in oracion.words]
        rev_lemma=" ".join(rev_lemma)
        rev_lemma_st.append(rev_lemma)

In [22]:
rev_lemma_stanza=[]
lemmatizar(review_tokens_unidos,rev_lemma_stanza)
data_lemma["rev_lemma_stanza"] = rev_lemma_stanza
data_lemma.head()

2021-07-30 13:27:40 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |

2021-07-30 13:27:40 INFO: Use device: cpu
2021-07-30 13:27:40 INFO: Loading: tokenize
2021-07-30 13:27:40 INFO: Loading: mwt
2021-07-30 13:27:40 INFO: Loading: pos
2021-07-30 13:27:41 INFO: Loading: lemma
2021-07-30 13:27:41 INFO: Done loading processors!
C:\Users\vanes\anaconda3\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_d

,stars,rev_lemma_nltk,review_tokens_unidos,rev_lemma_spacy,rev_lemma_stanza
0,1,malisimo muy grande demasiado aparatoso mal pr...,malisimo muy grande demasiado aparatoso mal pr...,malisimo mucho grande demasiado aparatoso mal ...,malisenar mucho grande demasiado aparatoso mal...
1,1,recibido pedido devolucion recibido,recibido pedido devolucion recibido,recibido pedido devolucion recibido,recibido pedido devolucion recibido
2,1,buscar sistema funciona muy bien abrazadera ll...,buscar sistema funciona muy bien abrazadera ll...,buscar sistema funcionar mucho bien abrazadera...,buscar sistema funcionar mucho bien abrazadera...
3,1,utilice broca menor diametro doblaron varias m...,utilice brocas menor diametro doblaron varias ...,utilizar broca menor diametro doblar varios ma...,utilizar broca menor diametro doblar varios ma...
4,1,gusta olor viejo aspecto malo gusta,gusta olor viejo aspecto malo gusta,gustar olor viejo aspecto malo gustar,gustar olor viejo aspecto malo gustar
